# Quantamental Database Explorer

Explore your Parquet-based data storage:
- **Parquet Files** (`data/factors/`) - All historical data (prices, factors, macro)
- **DuckDB** - SQL query interface over Parquet files

**No SQLite** - We use Parquet as the single source of truth for efficient analytics!


In [1]:
# Setup: imports and configuration
import sys
from pathlib import Path

import duckdb
import pandas as pd
from IPython.display import display

# Configure pandas to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Resolve paths (works from repo root or notebooks/)
CWD = Path.cwd().resolve()
CANDIDATES_ROOT = [CWD, CWD.parent]

for root in CANDIDATES_ROOT:
    if (root / "config" / "settings.py").exists():
        if str(root) not in sys.path:
            sys.path.insert(0, str(root))
        break

def resolve_first_existing(paths: list[Path], fallback: Path) -> Path:
    for p in paths:
        if p.exists():
            return p
    return fallback

PARQUET_DIR = resolve_first_existing(
    [root / "data" / "factors" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors",
)

DUCKDB_PATH = resolve_first_existing(
    [root / "data" / "factors" / "factors.duckdb" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors" / "factors.duckdb",
)

print(f"📁 Parquet Dir: {PARQUET_DIR}")
print(f"🦆 DuckDB: {DUCKDB_PATH}")
print(f"✅ Parquet dir exists: {PARQUET_DIR.exists()}")
print(f"✅ DuckDB exists: {DUCKDB_PATH.exists()}")


📁 Parquet Dir: /Users/andres/Downloads/Cursor/quant/data/factors
🦆 DuckDB: /Users/andres/Downloads/Cursor/quant/data/factors/factors.duckdb
✅ Parquet dir exists: True
✅ DuckDB exists: True


---
## 1. Parquet Files Overview
### Available Datasets


In [2]:
# List all Parquet files in the factors directory
import os

parquet_files = [f for f in os.listdir(PARQUET_DIR) if f.endswith('.parquet')]

print("Available Parquet Files:")
print("=" * 80)

file_info = []
for file in sorted(parquet_files):
    path = PARQUET_DIR / file
    size_mb = path.stat().st_size / (1024 * 1024)
    file_info.append({
        'File': file,
        'Size (MB)': f"{size_mb:.2f}",
        'Path': str(path)
    })

df_files = pd.DataFrame(file_info)
display(df_files)


Available Parquet Files:


,File,Size (MB),Path
0,factors_all.parquet,184.79,/Users/andres/Downloads/Cursor/quant/data/fact...
1,factors_price.parquet,184.79,/Users/andres/Downloads/Cursor/quant/data/fact...
2,macro.parquet,0.24,/Users/andres/Downloads/Cursor/quant/data/fact...
3,macro_z.parquet,1.02,/Users/andres/Downloads/Cursor/quant/data/fact...
4,prices.parquet,20.44,/Users/andres/Downloads/Cursor/quant/data/fact...


---
## 2. Prices Dataset
### Wide Format: Date × Symbols


In [3]:
# prices.parquet
prices_path = PARQUET_DIR / "prices.parquet"

if prices_path.exists():
    df_prices = pd.read_parquet(prices_path)
    
    print("=" * 80)
    print("prices.parquet - Wide format (date × tickers)")
    print("=" * 80)
    print(f"Shape: {df_prices.shape[0]:,} dates × {df_prices.shape[1]} tickers")
    print(f"Index: {df_prices.index.name}")
    print(f"Date range: {df_prices.index.min()} to {df_prices.index.max()}")
    print(f"\nFirst 10 tickers: {list(df_prices.columns[:10])}")
    print(f"Last 10 tickers: {list(df_prices.columns[-10:])}")
    
    print("\nFirst 5 rows:")
    display(df_prices.head(5))
    
    print("\nLast 5 rows:")
    display(df_prices.tail(5))
else:
    print(f"⚠️  prices.parquet not found at {prices_path}")


prices.parquet - Wide format (date × tickers)
Shape: 25,577 dates × 504 tickers
Index: date
Date range: 1927-12-30 00:00:00-05:00 to 2026-01-14 00:00:00-05:00

First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
Last 10 tickers: ['WTW', 'WDAY', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS', '^GSPC']

First 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1927-12-30 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Last 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2026-01-08 00:00:00-05:00,165.240005,70.400002,126.180000,224.130005,281.820007,339.040009,204.679993,14.22,110.260002,145.759995,261.220001,138.660004,85.709999,158.309998,53.270000,170.580002,164.270004,65.550003,211.100006,325.440002,326.010010,55.900002,246.289993,8.67,99.540001,115.930000,382.980011,77.900002,170.330002,129.270004,503.470001,210.039993,330.109985,136.250000,299.160004,351.970001,25.370001,146.190002,259.040009,281.640015,616.530029,86.449997,96.379997,61.189999,123.720001,264.540009,238.020004,24.170000,167.100006,276.579987,265.670013,3313.989990,184.080002,186.020004,612.979980,50.200001,54.830002,56.180000,20.690001,205.179993,499.769989,71.809998,64.580002,186.000000,1087.920044,155.300003,70.160004,120.050003,227.380005,5445.000000,98.379997,55.900002,332.480011,222.910004,80.589996,25.690001,111.290001,97.510002,67.050003,166.009995,318.799988,111.370003,26.459999,255.679993,202.490005,44.619999,31.690001,53.860001,608.130005,260.540009,165.960007,132.529999,335.739990,46.919998,38.000000,81.330002,215.509995,101.339996,209.710007,159.250000,39.180000,313.000000,85.860001,283.910004,165.190002,190.259995,73.959999,120.599998,61.230000,104.250000,266.640015,69.849998,69.370003,85.769997,245.589996,80.870003,28.209999,16.580000,98.720001,100.180000,147.960007,322.540009,83.129997,39.040001,85.330002,331.410004,70.139999,61.360001,915.309

---
## 3. Price Factors
### Momentum, Volatility, Beta


In [4]:
# factors_price.parquet
factors_price_path = PARQUET_DIR / "factors_price.parquet"

if factors_price_path.exists():
    df_factors_price = pd.read_parquet(factors_price_path)
    
    print("=" * 80)
    print("factors_price.parquet - Price-based factors (long format)")
    print("=" * 80)
    print(f"Shape: {df_factors_price.shape[0]:,} rows × {df_factors_price.shape[1]} columns")
    print(f"Index: {df_factors_price.index.names}")
    print(f"Columns: {list(df_factors_price.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_price.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_price.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_factors_price.describe())
else:
    print(f"⚠️  factors_price.parquet not found at {factors_price_path}")


factors_price.parquet - Price-based factors (long format)
Shape: 12,890,808 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2026-01-14 00:00:00-05:00 YUM     0.215628  0.033328  0.065811  0.262565   
                          ZBH    -0.161596 -0.007874 -0.067520  0.384707   
                          ZBRA   -0.366145 -0.198185 -0.094651  0.400859   
                          ZTS    -0.286458 -0.193335 -0.154723  0.365154   
                          ^GSPC   0.115885  0.083870  0.022155  0.112230   

factor                            beta_60d  
date                      symbol            
2026-01-14 00:00:00-05:00 YUM    -0.480336  
                          ZBH     0.152665  
                          ZBRA    1.768423  
                          ZTS     1.088427  
                          ^GSPC   1.000000


Descriptive statistics:


factor,mom_12_1,mom_6_1,mom_3_1,vol_60d,beta_60d
count,4.542132e+06,4.542132e+06,4.542132e+06,4.522476e+06,4.522476e+06
mean,1.459001e-01,6.476660e-02,2.563630e-02,3.091747e-01,9.280972e-01
std,4.673764e-01,2.687190e-01,1.632386e-01,2.759964e-01,6.823576e-01
min,-1.080000e+01,-1.227273e+01,-3.857143e+00,0.000000e+00,-1.426598e+01
25%,-6.257222e-02,-5.913334e-02,-4.695285e-02,1.949214e-01,5.751823e-01
50%,9.675270e-02,4.856224e-02,2.148003e-02,2.597273e-01,8.988693e-01
75%,2.806100e-01,1.625012e-01,9.148468e-02,3.621221e-01,1.222103e+00
max,5.218276e+01,3.252308e+01,2.539286e+01,5.398988e+01,1.282943e+02


---
## 4. All Factors Combined
### Price + Fundamental Factors


In [5]:
# factors_all.parquet
factors_all_path = PARQUET_DIR / "factors_all.parquet"

if factors_all_path.exists():
    df_factors_all = pd.read_parquet(factors_all_path)
    
    print("=" * 80)
    print("factors_all.parquet - All factors (price + fundamentals)")
    print("=" * 80)
    print(f"Shape: {df_factors_all.shape[0]:,} rows × {df_factors_all.shape[1]} columns")
    print(f"Index: {df_factors_all.index.names}")
    print(f"Columns: {list(df_factors_all.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_all.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_all.tail(5))
else:
    print(f"⚠️  factors_all.parquet not found at {factors_all_path}")


factors_all.parquet - All factors (price + fundamentals)
Shape: 12,890,808 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2026-01-14 00:00:00-05:00 YUM     0.215628  0.033328  0.065811  0.262565   
                          ZBH    -0.161596 -0.007874 -0.067520  0.384707   
                          ZBRA   -0.366145 -0.198185 -0.094651  0.400859   
                          ZTS    -0.286458 -0.193335 -0.154723  0.365154   
                          ^GSPC   0.115885  0.083870  0.022155  0.112230   

factor                            beta_60d  
date                      symbol            
2026-01-14 00:00:00-05:00 YUM    -0.480336  
                          ZBH     0.152665  
                          ZBRA    1.768423  
                          ZTS     1.088427  
                          ^GSPC   1.000000

---
## 5. Macroeconomic Indicators
### CPI, Unemployment, Fed Funds, Yields


In [6]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,621 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2026-01-14 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2026-01-08,0.026533,4.4,3.72,4.19,0.70
2026-01-09,0.026533,4.4,3.72,4.18,0.64
2026-01-12,0.026533,4.4,3.72,4.19,0.65
2026-01-13,0.026533,4.4,3.72,4.18,0.65
2026-01-14,0.026533,4.4,3.72,4.18,0.64



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20360.000000,20360.000000,18665.000000,16707.000000,12947.000000
mean,0.035289,5.662471,4.620842,5.822441,0.849464
std,0.028681,1.708061,3.570048,2.941359,0.917875
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016675,4.300000,1.930000,3.890000,0.190000
50%,0.028630,5.500000,4.320000,5.460000,0.790000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


---
## 6. Z-Scored Macro Indicators
### Standardized for Cross-Sectional Analysis


In [7]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,621 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2026-01-14 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2026-01-08,-0.930053,0.455523,0.180817,0.670695,0.864040
2026-01-09,-0.930074,0.459302,0.179592,0.660249,0.774842
2026-01-12,-0.930095,0.463118,0.178365,0.668241,0.792235
2026-01-13,-0.930116,0.466971,0.177136,0.657780,0.794351
2026-01-14,-0.930137,0.470862,0.175905,0.656550,0.781271



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20256.000000,20256.000000,18561.000000,16603.000000,12843.000000
mean,0.097648,-0.049027,0.284992,0.063563,-0.213155
std,1.344661,1.483989,1.485290,1.425762,1.359783
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.778690,-1.256427,-0.956603,-0.943674,-1.358180
50%,-0.167098,-0.376298,0.322258,-0.073761,-0.381916
75%,1.059871,0.901137,1.430104,1.180392,0.916504
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## 7. DuckDB Query Examples
### SQL Interface Over Parquet Files


In [8]:
# Connect to DuckDB and run SQL queries
if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH))
    
    # Recreate views with absolute paths (ensures queries work from any working directory)
    print("🔧 Recreating DuckDB views with absolute paths...")
    views = {
        'prices': PARQUET_DIR / 'prices.parquet',
        'macro': PARQUET_DIR / 'macro.parquet',
        'macro_z': PARQUET_DIR / 'macro_z.parquet',
        'factors_price': PARQUET_DIR / 'factors_price.parquet',
        'factors_all': PARQUET_DIR / 'factors_all.parquet',
    }
    
    for name, path in views.items():
        if path.exists():
            con.execute(f"CREATE OR REPLACE VIEW {name} AS SELECT * FROM read_parquet('{path.as_posix()}')")
            print(f"   ✅ {name}")
        else:
            print(f"   ⚠️  {name} - file not found at {path}")
    print()
    
    print("=" * 80)
    print("DuckDB Query Examples")
    print("=" * 80)
    
    # Example 1: Get AAPL prices for recent dates
    print("\n1. Recent AAPL prices:")
    query1 = """
    SELECT * FROM prices 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    # Note: prices is in wide format, so this won't work as-is
    # Let's query factors instead
    
    # Example 2: Get factors for AAPL
    print("\n2. Recent AAPL factors:")
    query2 = """
    SELECT * FROM factors_price 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    try:
        result = con.sql(query2).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    # Example 3: Top momentum stocks
    print("\n3. Top 10 momentum stocks (most recent date):")
    query3 = """
    WITH latest_date AS (
        SELECT MAX(date) as max_date FROM factors_price
    )
    SELECT symbol, mom_12_1, mom_6_1, mom_3_1
    FROM factors_price
    WHERE date = (SELECT max_date FROM latest_date)
    AND mom_12_1 IS NOT NULL
    ORDER BY mom_12_1 DESC
    LIMIT 10
    """
    try:
        result = con.sql(query3).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    con.close()
else:
    print(f"⚠️  DuckDB not found at {DUCKDB_PATH}")


🔧 Recreating DuckDB views with absolute paths...
   ✅ prices
   ✅ macro
   ✅ macro_z
   ✅ factors_price
   ✅ factors_all

DuckDB Query Examples

1. Recent AAPL prices:

2. Recent AAPL factors:


,mom_12_1,mom_6_1,mom_3_1,vol_60d,beta_60d,date,symbol
0,0.213921,0.282609,0.094214,0.140412,0.516677,2026-01-13 23:00:00-06:00,AAPL
1,0.228203,0.304893,0.115514,0.161830,0.627295,2026-01-12 23:00:00-06:00,AAPL
2,0.232877,0.302292,0.114786,0.166362,0.654532,2026-01-11 23:00:00-06:00,AAPL
3,0.197419,0.309977,0.127112,0.167176,0.662632,2026-01-08 23:00:00-06:00,AAPL
4,0.191804,0.307128,0.085127,0.167570,0.672490,2026-01-07 23:00:00-06:00,AAPL
5,0.203688,0.296047,0.071987,0.167143,0.670578,2026-01-06 23:00:00-06:00,AAPL
6,0.161855,0.294058,0.081825,0.167130,0.657462,2026-01-05 23:00:00-06:00,AAPL
7,0.193542,0.313676,0.089058,0.178133,0.808943,2026-01-04 23:00:00-06:00,AAPL
8,0.202429,0.336706,0.096588,0.178783,0.849361,2026-01-01 23:00:00-06:00,AAPL
9,0.197887,0.344951,0.107358,0.178907,0.852214,2025-12-31 23:00:00-06:00,AAPL



3. Top 10 momentum stocks (most recent date):


,symbol,mom_12_1,mom_6_1,mom_3_1
0,WDC,2.891889,1.912055,0.535412
1,WBD,1.821371,1.281842,0.588113
2,STX,1.786786,1.001361,0.329972
3,MU,1.776028,1.510574,0.333162
4,NEM,1.576047,0.816628,0.074762
5,HOOD,1.338029,0.052225,-0.146200
6,LRCX,1.299571,0.803425,0.171334
7,PLTR,1.285266,0.188530,0.019674
8,DG,0.980660,0.235591,0.284210
9,INTC,0.959839,0.810237,0.012586


---
## Summary

**Parquet-Based Architecture:**
- **Single Source of Truth**: All data in Parquet files (no SQLite)
- **Wide Format**: prices.parquet (date × symbols)
- **Long Format**: factors (date, symbol, factors)
- **DuckDB**: SQL query layer (no storage, just views)

**Incremental Updates:**
- Run `python scripts/update_daily.py` to fetch new data
- Run `python scripts/add_symbol.py TICKER` to add stocks
- Cron job runs daily at 6 PM automatically


In [9]:
# Summary statistics
print("=" * 80)
print("DATA SUMMARY")
print("=" * 80)

summary = []

for file in ['prices', 'factors_price', 'factors_all', 'macro', 'macro_z']:
    path = PARQUET_DIR / f"{file}.parquet"
    if path.exists():
        df = pd.read_parquet(path)
        size_mb = path.stat().st_size / (1024 * 1024)
        
        summary.append({
            'Dataset': file,
            'Rows': f"{df.shape[0]:,}",
            'Columns': df.shape[1],
            'Size (MB)': f"{size_mb:.2f}",
            'Date Range': f"{df.index.get_level_values(0).min() if isinstance(df.index, pd.MultiIndex) else df.index.min()} to {df.index.get_level_values(0).max() if isinstance(df.index, pd.MultiIndex) else df.index.max()}"
        })

df_summary = pd.DataFrame(summary)
display(df_summary)

print("\n✅ All datasets ready for quantamental research!")
print("📚 See INCREMENTAL_UPDATES.md for architecture details")
print("🔄 Run 'python scripts/update_daily.py' to update data")


DATA SUMMARY


,Dataset,Rows,Columns,Size (MB),Date Range
0,prices,"25,577",504,20.44,1927-12-30 00:00:00-05:00 to 2026-01-14 00:00:...
1,factors_price,"12,890,808",5,184.79,1927-12-30 00:00:00-05:00 to 2026-01-14 00:00:...
2,factors_all,"12,890,808",5,184.79,1927-12-30 00:00:00-05:00 to 2026-01-14 00:00:...
3,macro,"20,621",5,0.24,1947-01-01 00:00:00 to 2026-01-14 00:00:00
4,macro_z,"20,621",5,1.02,1947-01-01 00:00:00 to 2026-01-14 00:00:00



✅ All datasets ready for quantamental research!
📚 See INCREMENTAL_UPDATES.md for architecture details
🔄 Run 'python scripts/update_daily.py' to update data


### `macro.parquet` - Macroeconomic indicators


In [10]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,621 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2026-01-14 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2026-01-08,0.026533,4.4,3.72,4.19,0.70
2026-01-09,0.026533,4.4,3.72,4.18,0.64
2026-01-12,0.026533,4.4,3.72,4.19,0.65
2026-01-13,0.026533,4.4,3.72,4.18,0.65
2026-01-14,0.026533,4.4,3.72,4.18,0.64



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20360.000000,20360.000000,18665.000000,16707.000000,12947.000000
mean,0.035289,5.662471,4.620842,5.822441,0.849464
std,0.028681,1.708061,3.570048,2.941359,0.917875
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016675,4.300000,1.930000,3.890000,0.190000
50%,0.028630,5.500000,4.320000,5.460000,0.790000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


### `macro_z.parquet` - Z-scored macro indicators


In [11]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,621 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2026-01-14 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2026-01-08,-0.930053,0.455523,0.180817,0.670695,0.864040
2026-01-09,-0.930074,0.459302,0.179592,0.660249,0.774842
2026-01-12,-0.930095,0.463118,0.178365,0.668241,0.792235
2026-01-13,-0.930116,0.466971,0.177136,0.657780,0.794351
2026-01-14,-0.930137,0.470862,0.175905,0.656550,0.781271



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20256.000000,20256.000000,18561.000000,16603.000000,12843.000000
mean,0.097648,-0.049027,0.284992,0.063563,-0.213155
std,1.344661,1.483989,1.485290,1.425762,1.359783
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.778690,-1.256427,-0.956603,-0.943674,-1.358180
50%,-0.167098,-0.376298,0.322258,-0.073761,-0.381916
75%,1.059871,0.901137,1.430104,1.180392,0.916504
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## Summary

**Database Architecture:**
- **SQLite** (`stock_data.db`) - Operational cache for recent data
- **Parquet** files - Analytics-optimized historical datasets
  - `prices.parquet` - Wide format (date × tickers)
  - `factors_price.parquet` - Long format (date, symbol, factors)
  - `factors_all.parquet` - Combined price + fundamental factors
  - `macro.parquet` - Raw macroeconomic indicators
  - `macro_z.parquet` - Standardized macro indicators
